# 卷积神经网络

In [1]:
import numpy as np

说明: 本笔记是邱锡鹏老师[《神经网络与深度学习》](https://nndl.github.io)第五章1、2、3小节的总结。

卷积神经网络是一种具有**局部连接**、**权重共享**等特性的深层[前馈神经网络](fnn.ipynb)。

目前的卷积神经网络一般是由卷积层、汇聚层和全连接层交叉堆叠而成的前馈神经网络，使用反向传播算法进行训练

## Convolution

### 一维卷积

我们称$w_1, w_2, \ldots, w_K$为滤波器(Filter)，它和一个信号序列$x_1, x_2, \ldots$的卷积为
$$
y_t = \displaystyle\sum_{k = 1}^K w_kx_{t - k  + 1}\tag{1}
$$

那么，我们可以定义信号序列$x$和滤波器$w$的卷积定义为
$$
y = w * x\tag{2}
$$
一般情况下，滤波器的长度$K$远远小于信号序列的长度$N$。

In [2]:
# 使用numpy做一个一维卷积的实验
w = np.array([-1, 0, 1])
x = np.random.randint(-1, 3, size=(10))

def conv1(kernel: np.array, data: np.array):
    kernel = kernel[::-1]
    result_len = len(data) - len(kernel) + 1
    return np.array([np.sum(kernel * data[i: i + kernel.shape[0]])
                     for i in range(result_len)])

In [3]:
print("Filter is {}".format(w))
print("data sequence is {}".format(x))
print("Result is {}".format(conv1(w, x)))

Filter is [-1  0  1]
data sequence is [-1  1 -1  2  1  0  0 -1  0  2]
Result is [ 0 -1 -2  2  1  1  0 -3]


### 二维卷积

卷积也经常在图像处理中使用，图像是二维结构，所以需要讲以为卷积进行拓展，给定一个图像$X\in R^{M\times N}$和滤波器$W\in R^{U\times V}$，其卷积为
$$
y_{ij} = \displaystyle\sum_{u = }^U\sum_{v = 1}^V w_{uv}w_{i - u + 1, j - v + 1}\tag{3}
$$

一个输入信息$X$和滤波器$W$的二维卷积定义为
$$
Y = W * X\tag{4}
$$

In [4]:
w = np.arange(4).reshape(2, 2)
x = np.arange(16).reshape(4, 4)

def conv2(kernel: np.array, data: np.array):
    kernel = kernel[::-1, ::-1]
    dim0 = data.shape[0] - kernel.shape[0] + 1
    dim1 = data.shape[1] - kernel.shape[1] + 1
    return np.array([[np.sum(kernel * data[i: i + kernel.shape[0], 
                                    j: j + kernel.shape[1]]) 
               for j in range(dim1)] 
              for i in range(dim0)])

In [5]:
print("The kernel is: ")
print(w)
print("The 2D data is: ")
print(x)
print("The result is:")
print(conv2(w, x))

The kernel is: 
[[0 1]
 [2 3]]
The 2D data is: 
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
The result is:
[[ 6 12 18]
 [30 36 42]
 [54 60 66]]


图像处理中常用的均值滤波器就是一种二维卷积, 它的滤波器元素$w_{uv} = \frac{1}{UV}$

### 互相关

观察上述代码实现卷积计算的过程中，需要进行卷积核翻转。 在实际代码中，一般会以互相关操作来代替卷 积。互相关是一个衡量两个序列相关性的函数，通常是用滑动窗口的点积计算来实现
$$
y_{ij} = \displaystyle\sum_{u = }^U\sum_{v = 1}^V w_{uv}w_{i + u - 1, j + v - 1}\tag{5}
$$

和公式(3)对比可知，互相关和卷积的区别在于卷积核是否翻转，因此互相关也可以称为不翻转卷积公式(5)可以表述为
$$
\begin{aligned}
Y =& W\otimes X\\
=& \text{rot180}(W) * X
\end{aligned}
\tag{6}
$$

在神经网络中使用卷积是为了进行特征抽取，卷积核是否进行翻转和其特 征抽取的能力无关。特别是当卷积核是可学习的参数时，卷积和互相关在能力上是等价的。笔记后续的卷积全部指互相关。

### 卷积变种

假设卷积层的输入神经元个数为$M$，卷积大小为$K$，步长为$S$，在输入两端各填补$P$个0(zero padding)，那么该卷积层的神经元数量为$(M - K + 2P)/S + 1$


一般常用的卷积有一下三类

1. 窄卷积：步长$S = 1$，两端不补零$P = 0$，卷积后输出长度为$M - K + 1$

2. 宽卷积：步长$S = 1$，两端补零$P = K - 1$，卷积后输出长度$M + K - 1$

3. 等宽卷积：步长$S = 1$，两端补零$P = (K - 1)/2$, 卷积后输出长度$M$

### 卷积的数学性质

对图像两端各补$U - 1$和$V - 1$个零，得到全填充的图像$\tilde{X}$, 图像$X$和卷积核$W$的宽卷积定义为
$$
W \tilde{\otimes}X \overset{\Delta}{=} W\otimes \tilde{X}\tag{7}
$$
其中$\tilde{\otimes}$表示宽卷积运算.

当输入信息和卷积核有固定长度时，它们的宽卷积依然具有交换性，即
$$
\text{rot180}(W)\tilde{\otimes}X \overset{\Delta}{=}\text{rot180}(X)\tilde{\otimes}W\tag{8}
$$

假设$Y = W\otimes X$，函数$f(Y) \in R$为一个标量函数，则
$$
\begin{aligned}
\frac{\partial f(Y)}{\partial w_{uv}} =&
\displaystyle\sum_{i = 1}^{M - U + 1}\sum_{j = 1}^{N - V + 1}\frac{\partial y_{ij}}{\partial w_{uv}}\frac{\partial f(Y)}{\partial y_{ij}}\\
=& \displaystyle\sum_{i = 1}^{M - U + 1}\sum_{j = 1}^{N - V + 1}x_{i + u -1, j + v - 1} \frac{\partial f(Y)}{\partial y_{ij}}\\
\end{aligned}\tag{9}
$$

从公式(9)可以看出，$f(Y)$关于$W$的偏导数为
$$
\frac{\partial f(Y)}{\partial W} = \frac{\partial f(Y)}{\partial Y} \otimes X\tag{10}
$$

同理得到
$$
\begin{aligned}
\frac{\partial f(Y)}{\partial x_{st}} =&
\displaystyle\sum_{i = 1}^{M - U + 1}\sum_{j = 1}^{N - V + 1}\frac{\partial y_{ij}}{\partial x_{st}}\frac{\partial f(Y)}{\partial y_{ij}}\\
=& \displaystyle\sum_{i = 1}^{M - U + 1}\sum_{j = 1}^{N - V + 1}w_{s - i +1, t 
- j + 1} \frac{\partial f(Y)}{\partial y_{ij}}\\
\end{aligned}\tag{11}
$$

当其中$(s - i + 1) < 1$或者$(s - i  + 1) > U$, $(t - j + 1) < 1$或者$(t - j + 1) > V$时, $w_{s - i + 1, t - j + 1} = 0$, 即相当于对$W$进行了$P = (M - U, N - V)$的零填充。

从公式(11)可以得到
$$
\begin{aligned}
\frac{\partial f(Y)}{\partial X} =&
\text{rot180}(\frac{\partial f(Y)}{\partial Y})\tilde{\otimes}W\\
=& \text{rot180}(W)\tilde{\otimes} \frac{\partial f(Y)}{\partial Y}\\
\end{aligned}\tag{12}
$$

## Convolution Neural Network

### 卷积层

由于卷积网络主要应用在图像处理上, 而图像为二维结构，因此为了更充分地利用 图像的局部信息，通常将神经元组织为三维结构的神经层，其大小为高度$M\times$宽度$N\times$深度$D$，由$D$个$M\times N$大小的特征映射构成.

特征映射为一幅图像（或其他特征映射）在经过卷积提取到的特征，每个特征映射可以作为一类抽取的图像特征. 为了提高卷积网络的表 示能力，可以在每一层使用多个不同的特征映射，以更好地表示图像的特征.

在输入层，特征映射就是图像本身. 如果是灰度图像，就是有一个特征映射， 输入层的深度 𝐷 = 1；如果是彩色图像，分别有 RGB 三个颜色通道的特征映射，输入层的深度$D = 3$

不失一般性，假设一个卷积层的结构如下：

1. 输入特征映射组：$X \in R^{M\times N\times D}$为三维张量，其中每个切片矩阵$X^d \in R^{M\times N}$为一个输入特征映射，$1\le d\le D$
2. 输出特征映射组：$Y \in R^{M’\times N‘\times P}$为三维张量，其中每个切片矩阵$Y^p \in R^{M'\times N'}$为一个输入特征映射，$1\le p\le P$
3. 卷积核：$W\in R^{U\times V\times D\times P}$为四维张量，其中每个切片矩阵$W_{p, d}\in R^{U\times V}$

为了计算输出特征映射$Y^p$ ，用卷积核$W^{p, 1},W^{p, 2}, \ldots,W^{p, D}$ 分别对输入特征映射$X^1, X^2, \ldots, X^D$进行卷积，然后将卷积结果相加，并加上一个标量偏置$b$得到卷积层的净输入$Z^p$ 𝒁 𝑝 ，再经过非线性激活函数后得到输出特征映射$Y^p$.

$$
Z^p = W^p\otimes X + b^p = \displaystyle\sum_{d = 1}^DW^{p, d}\otimes X^d + b^p\tag{13}
$$
$$
Y^p = f(Z^p)\tag{14}
$$

其中$W^p\in R^{U\times V\times D}$为三维卷积核， $f(\cdot)$为非线性函数，一般选用ReLU.

如果希望卷积层输出$P$个特征映射，可以将上述过程重复$P$次。

### 汇聚层

汇聚层也叫子采样层，其作用是进行特征选择，降低特征数量，从而减少参数数量.

卷积层虽然可以显著减少网络中连接的数量，但特征映射组中的神经元个 数并没有显著减少. 如果后面接一个分类器，分类器的输入维数依然很高，很容 易出现过拟合. 为了解决这个问题，可以在卷积层之后加上一个汇聚层，从而降 低特征维数，避免过拟合.

常用的汇聚函数有两种
1. 最大汇聚： 对于一个区域$R^d_{m,n}$，选择这个区域最大值作为这个区域的代表
  $$y^d_{m,n} = \underset{i\in R^d{m,n}}{\max}x_i\tag{15}$$
2. 平均汇聚：对于一个区域$R^d_{m,n}$，选择这个区域平均值作为这个区域的代表
  $$y^d_{m,n} = \frac{1}{|R^d_{m,n}|}\underset{i\in R^d{m,n}}{\sum}x_i\tag{16}$$

对每一个输入特征映射$X^d$的$M'\times N'$个区域进行子采样，得到汇聚层的输出特征映射$Y^d = \{y^d_{m,n}\}$, $1\le m \le M'$, $1\le n \le N'$

## 参数学习

在卷积神经网络中，主要有两种不同功能的神经层：卷积层和汇聚层. 而参 数为卷积核以及偏置，因此只需要计算卷积层中参数的梯度.

不失一般性，对第$l$层为卷积层，第$l-1$层的输入特征映射为$X^{(l-1)}\in R^{M\times N\times D}$，通过卷积计算得到第$l$层的特征映射净输入$Z^{l}\in R^{M'\times N'\times P}$ 第$l$层 的第$p(1\le p\le P)$个特征映射净输入。

$$
Z^{(l, p)} = \displaystyle\sum_{d = 1}^DW^{(l, p, d)}\otimes X^{(l-1, d)} + b^{(l, p)}\tag{17}
$$

根据公式(10)和(12)，模型损失$Loss$关于第$l$层的卷积核$W^{(l, p, d)}$的偏导数为
$$
\begin{aligned}
\frac{\partial Loss}{\partial W^{(l, p, d)}} =&\frac{\partial Loss}{\partial Z^{(l, p)}}\otimes X^{(l-1, d)}\\
=& \delta^{(l, p)}\otimes X^{(l - 1, d)}
\end{aligned}\tag{18}
$$

其中$\delta^{(l, p)} = \frac{\partial Loss}{\partial Z^{(l, p)}}$。

同理，模型损失关于第$l$层的第$p$个偏置$b^{(l,p)}$的偏导数为
$$
\frac{\partial Loss}{\partial b^{(l, p)}} = \displaystyle\sum_{i, j}\left[\delta^{(l,p)}\right]_{i, j}\tag{19}
$$

### 误差项的计算

卷积层和汇聚层中误差项的计算有所不同，因此我们分别计算其误差项.

当第$l + 1$层为**汇聚层**时，因为汇聚层是下采样操作，$l + 1$层的每个神经元的误差项$\delta$对应于第$l$层的相应特征映射的一个区域。也就是说第$l$层输出的每一层和第$l + 1$层的输出具有函数关系。

第$l$层的第$p$个特征映射的误差项$\delta^{(l, p)}$的推导过程如下
$$
\begin{aligned}
\delta^{(l, p)}
=& \frac{\partial Loss}{\partial Z^{(l, p)}}\\
=& \frac{\partial X^{(l, p)}}{\partial Z^{(l,p)}}\frac{\partial Z^{(l + 1, p)}}{\partial X^{(l, p)}}\frac{\partial Loss}{\partial Z^{(l + 1, p)}}\\
=& f_l'(Z^{(l, p)})\odot \text{up}(\delta^{(l + 1, p)})
\end{aligned}\tag{20}
$$

其中$f_l'(\cdot)$为第$l$层使用的激活函数导数, up为上采样函数, 与汇 聚层中使用的下采样操作刚好相反. 如果下采样是最大汇聚，误差项$\delta^{(l + 1, p)}$中 每个值会直接传递到上一层对应区域中的最大值所对应的神经元，该区域中其 他神经元的误差项都设为 0. 如果下采样是平均汇聚，误差项$\delta^{(l + 1, p)}$中每个值会 被平均分配到上一层对应区域中的所有神经元上.

当第$l + 1$层为**卷积层**时, 假设特征映射净输入$Z^{(l + 1, p)} \in R^{M'\times N'\times P}$，那么，改写公式(17)
$$
Z^{(l + 1, p)} = \displaystyle\sum_{d = 1}^DW^{(l + 1, p, d)}\otimes X^{(l, d)} + b^{(l + 1, p)}\tag{21}
$$

第$l$层的第$d$个特征映射的误差项$\delta^{(l, d)}$的具体推导过程如下：
$$
\begin{aligned}
\delta^{(l, d)} 
=& \frac{\partial Loss}{\partial Z^{(l, d)}}\\
=& \frac{\partial X^{(l, d)}}{\partial Z^{(l, d)}}\frac{\partial Loss}{\partial X^{(l, d)}}\\
=& f_l'(Z^{(l, d)})\odot\displaystyle\sum_{p = 1}^P\left(\text{rot180}(W^{(l + 1, p, d)})\tilde{\otimes}\frac{\partial Loss}{\partial Z^{(l + 1, p)}}\right) \\
=& f_l'(Z^{(l, d)})\odot\displaystyle\sum_{p = 1}^P\left(\text{rot180}(W^{(l + 1, p, d)})\tilde{\otimes}\delta^{(l + 1, p)} \right) \\
\end{aligned}
$$